# Invoice Data Extraction & Analysis Demo

This notebook demonstrates the complete invoice extraction and analysis pipeline using OpenAI's GPT-4 Vision API.

## Setup & Configuration

In [ ]:
# Import required libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from dotenv import load_dotenv

# Import our custom modules
from invoice_extractor import InvoiceExtractor
from query_engine import InvoiceQueryEngine

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")

## Step 1: Extract Data from Invoices

In [ ]:
# Initialize the extractor
extractor = InvoiceExtractor()

print("✓ Invoice extractor initialized")

In [ ]:
# Process all invoices in the data folder
invoice_folder = "data/invoices"

print(f"Processing invoices from: {invoice_folder}\n")
results = extractor.process_folder(invoice_folder)

print(f"\n✓ Processed {len(results['invoices'])} invoices")

In [ ]:
# Save to CSV files
output_dir = "output"
extractor.save_to_csv(results, output_dir=output_dir)

print(f"✓ Data saved to {output_dir}/")

## Step 2: Explore the Extracted Data

In [ ]:
# Load the invoices data
invoices_df = results['invoices']
line_items_df = results['line_items']

print(f"Invoices: {len(invoices_df)} records")
print(f"Line Items: {len(line_items_df)} records")

In [ ]:
# Display first few invoices
print("\n=== SAMPLE INVOICES ===")
invoices_df.head()

In [ ]:
# Display invoice schema
print("\n=== INVOICE SCHEMA ===")
invoices_df.info()

In [ ]:
# Display first few line items
print("\n=== SAMPLE LINE ITEMS ===")
line_items_df.head()

In [ ]:
# Basic statistics
print("\n=== BASIC STATISTICS ===")
print(f"Total Amount: ${invoices_df['total_amount'].sum():,.2f}")
print(f"Average Invoice: ${invoices_df['total_amount'].mean():,.2f}")
print(f"Median Invoice: ${invoices_df['total_amount'].median():,.2f}")
print(f"Unique Vendors: {invoices_df['vendor_name'].nunique()}")
print(f"Date Range: {invoices_df['date'].min()} to {invoices_df['date'].max()}")

## Step 3: Data Quality Analysis

In [ ]:
# Check for missing values
print("\n=== MISSING VALUES ===")
missing = invoices_df.isnull().sum()
print(missing[missing > 0])

In [ ]:
# Check confidence scores
print("\n=== EXTRACTION CONFIDENCE ===")
print(invoices_df['confidence'].value_counts())

In [ ]:
# Check for errors
if 'error' in invoices_df.columns:
    errors = invoices_df[invoices_df['error'].notna()]
    print(f"\nFound {len(errors)} extraction errors:")
    print(errors[['source_file', 'error']])
else:
    print("✓ No extraction errors found")

## Step 4: Query and Analysis

In [ ]:
# Initialize query engine
engine = InvoiceQueryEngine(
    "output/invoices.csv",
    "output/line_items.csv"
)

print("✓ Query engine initialized")

In [ ]:
# Get summary statistics
stats = engine.get_invoice_summary_stats()

print("\n=== SUMMARY STATISTICS ===")
for key, value in stats.items():
    if isinstance(value, dict):
        print(f"\n{key}:")
        for k, v in value.items():
            print(f"  {k}: {v}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Total spend by vendor
vendor_spend = engine.get_total_spend_by_vendor()

print("\n=== TOTAL SPEND BY VENDOR ===")
vendor_spend.head(10)

In [ ]:
# Monthly spending trend
monthly_spend = engine.get_total_spend_by_month()

print("\n=== MONTHLY SPENDING ===")
monthly_spend

## Step 5: Visualizations

In [ ]:
# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Spend by vendor - Bar chart
fig, ax = plt.subplots(figsize=(12, 6))

top_vendors = vendor_spend.head(10)
top_vendors.plot(kind='barh', y='total_spend', ax=ax, color='steelblue')

ax.set_xlabel('Total Spend ($)', fontsize=12)
ax.set_ylabel('Vendor', fontsize=12)
ax.set_title('Top 10 Vendors by Total Spend', fontsize=14, fontweight='bold')
ax.legend().remove()

plt.tight_layout()
plt.show()

In [ ]:
# Monthly spending trend - Line chart
fig, ax = plt.subplots(figsize=(12, 6))

monthly_df = monthly_spend.reset_index()
monthly_df['month'] = monthly_df['month'].astype(str)

ax.plot(monthly_df['month'], monthly_df['total_spend'], 
        marker='o', linewidth=2, markersize=8, color='steelblue')

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Spend ($)', fontsize=12)
ax.set_title('Monthly Spending Trend', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Invoice amount distribution
fig, ax = plt.subplots(figsize=(12, 6))

invoices_df['total_amount'].hist(bins=20, ax=ax, color='steelblue', edgecolor='black')

ax.set_xlabel('Invoice Amount ($)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Invoice Amount Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## Step 6: Custom Queries

In [ ]:
# Example: Get invoices from a specific vendor
# Replace 'Vendor Name' with an actual vendor from your data

vendors = engine.get_vendor_list()
print("Available vendors:")
for vendor in vendors[:5]:  # Show first 5
    print(f"  - {vendor}")

# Uncomment and modify to query a specific vendor:
# vendor_invoices = engine.get_invoices_by_vendor("Acme Corp")
# vendor_invoices

In [ ]:
# Example: Get invoices in a date range
# date_range_invoices = engine.get_invoices_by_date_range(
#     "2025-03-01", 
#     "2025-06-30"
# )
# print(f"Found {len(date_range_invoices)} invoices in the date range")
# date_range_invoices

In [ ]:
# Example: Search for specific items
# search_results = engine.search_line_items("software")
# print(f"Found {len(search_results)} line items matching 'software'")
# search_results

In [ ]:
# Example: Get details for a specific invoice
# invoice_details = engine.get_invoice_details(1)
# print("\nInvoice:")
# print(invoice_details['invoice'])
# print("\nLine Items:")
# for item in invoice_details['line_items']:
#     print(item)

## Step 7: Validation & Accuracy Check

In [ ]:
# Validate that line items sum to invoice total
validation_results = []

for invoice_id in invoices_df['invoice_id'].unique():
    invoice = invoices_df[invoices_df['invoice_id'] == invoice_id].iloc[0]
    line_items = line_items_df[line_items_df['invoice_id'] == invoice_id]
    
    calculated_total = line_items['line_total'].sum()
    declared_total = invoice['total_amount']
    
    difference = abs(calculated_total - declared_total)
    
    validation_results.append({
        'invoice_id': invoice_id,
        'invoice_number': invoice['invoice_number'],
        'declared_total': declared_total,
        'calculated_total': calculated_total,
        'difference': difference,
        'valid': difference < 0.01  # Allow 1 cent tolerance
    })

validation_df = pd.DataFrame(validation_results)

print("\n=== VALIDATION RESULTS ===")
print(f"Total invoices: {len(validation_df)}")
print(f"Valid: {validation_df['valid'].sum()}")
print(f"Invalid: {(~validation_df['valid']).sum()}")

# Show invalid invoices
invalid = validation_df[~validation_df['valid']]
if len(invalid) > 0:
    print("\nInvalid invoices (totals don't match):")
    print(invalid)

## Step 8: Export Reports

In [ ]:
# Create a comprehensive report
report_data = {
    'Metric': [
        'Total Invoices',
        'Total Amount',
        'Average Invoice',
        'Unique Vendors',
        'Total Line Items',
        'Valid Extractions',
    ],
    'Value': [
        len(invoices_df),
        f"${invoices_df['total_amount'].sum():,.2f}",
        f"${invoices_df['total_amount'].mean():,.2f}",
        invoices_df['vendor_name'].nunique(),
        len(line_items_df),
        validation_df['valid'].sum(),
    ]
}

report_df = pd.DataFrame(report_data)
print("\n=== EXTRACTION REPORT ===")
print(report_df.to_string(index=False))

# Save report
report_df.to_csv('output/extraction_report.csv', index=False)
print("\n✓ Report saved to output/extraction_report.csv")

## Conclusion

This notebook demonstrated:
1. ✓ Automated extraction from invoice documents using GPT-4 Vision
2. ✓ Data normalization and storage in CSV format
3. ✓ Query capabilities for business analysis
4. ✓ Visualization of spending patterns
5. ✓ Data validation and quality checks

### Next Steps:
- Run `streamlit run dashboard.py` for interactive visualization
- Adjust extraction prompts for better accuracy
- Add more validation rules
- Scale to larger invoice datasets